In [11]:
# بسم الله الرحمن الرحيم وبه نستعين
# We import dataclass to represent 1 gene 
from dataclasses import dataclass
import random
from deap import base, creator, tools, algorithms

In [12]:
# A gene contains 2 attributes : the food item's name and the number of servings of this item
@dataclass
class Gene:
    food_Item_Name: str
    serving: int

In [13]:
# We make a list representing all the meals we have in 1 chromosome 
Meals =["Breakfast", "Snack1", "Lunch", "Snack2", "Dinner"]
# We make a list representing all the food groups we have in 1 meal
Food_Groups =["Vegetables", "Fruits", "Grains", "Protein", "Dairy", "Fats"]

In [14]:
# 1600 kcal Serving (taken from saudi calorie guide)
SERVING_GUIDELINES = {
    "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats": 1},
    "Snack1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
    "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats": 1},
    "Snack2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats": 0},
    "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats": 0}
}

In [15]:
# Detailed dataset with grams per serving (Protein, Fat, Carbs, Calories)
FOOD_DATA = {
    "Vegetables": {
        "Green Beans": {"p": 2.0, "f": 0.1, "c": 7.8, "cal": 34.1},
        "Potato":      {"p": 4.3, "f": 0.2, "c": 37.2, "cal": 164.0}
    },
    "Fruits": {
        "Blueberries": {"p": 1.1, "f": 0.5, "c": 21.0, "cal": 82.6},
        "Apple":       {"p": 0.4, "f": 0.2, "c": 19.1, "cal": 71.8}
    },
    "Grains": {
        "Rice (cooked)": {"p": 4.25, "f": 0.44, "c": 44.51, "cal": 205.4},
        "Salted Cracker":{"p": 0.22, "f": 0.76, "c": 1.83, "cal": 15.06}
    },
    "Protein": {
        "Omelet":        {"p": 6.5, "f": 7.3, "c": 0.4, "cal": 95.8},
        "Canned Chicken":{"p": 30.9, "f": 11.3, "c": 0.0, "cal": 234.3}
    },
    "Dairy": {
        "Grated Parmesan":{"p": 1.9, "f": 1.4, "c": 0.2, "cal": 21.6},
        "Cheddar Cheese": {"p": 7.0, "f": 9.3, "c": 0.4, "cal": 112.8}
    },
    "Fats": {
        "Corn Oil":    {"p": 0.0, "f": 13.64, "c": 0.0, "cal": 120.55},
        "Mayonnaise":  {"p": 0.13, "f": 4.91, "c": 3.51, "cal": 57.29}
    }
}

In [16]:
# DEAP Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_individual():
    # Initializes 30 genes following 1600 kcal
    genes = []
    for meal in Meals:
        for group in Food_Groups:
            # Get the fixed serving count for this meal/group
            serving_count = SERVING_GUIDELINES[meal][group]

            # Randomly select food from dataset
            food_name = random.choice(list(FOOD_DATA[group].keys()))

            # Create gene with guideline serving count
            genes.append(Gene(food_Item_Name=food_name, serving=serving_count))

    return creator.Individual(genes)

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Mutation
def mutate_meal_plan(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            group_index = i % 6
            group_name = Food_Groups[group_index]

            # Change the food, but the serving remains the same
            individual[i].food_Item_Name = random.choice(list(FOOD_DATA[group_name].keys()))
    return individual,


def evaluate_meal_plan(individual):
    total_p = 0.0
    total_f = 0.0
    total_c = 0.0

    # Sum up all grams based on food item and serving count
    for gene in individual:
        group_name = None
        # Find which group this food belongs to
        for group, foods in FOOD_DATA.items():
            if gene.food_Item_Name in foods:
                group_name = group
                break

        if group_name:
            stats = FOOD_DATA[group_name][gene.food_Item_Name]
            total_p += stats['p'] * gene.serving
            total_f += stats['f'] * gene.serving
            total_c += stats['c'] * gene.serving

    # Calculate percentages (p_c, p_p, p_f)
    g_total = total_p + total_f + total_c
    if g_total == 0: return 0, # Avoid division by zero

    p_p = total_p / g_total
    p_f = total_f / g_total
    p_c = total_c / g_total

    # Target Ratios
    r_p, r_f, r_c = 0.20, 0.25, 0.55

    # Calculate J_macro (Squared Error)
    j_macro = (p_c - r_c)**2 + (p_p - r_p)**2 + (p_f - r_f)**2

    # We want to minimize J_macro, so we return a fitness that increases as J_macro decreases
    return 1 / (1 + j_macro),


toolbox.register("evaluate", evaluate_meal_plan)
toolbox.register("mate", tools.cxTwoPoint) # One/Two-point meal-level crossover
toolbox.register("select", tools.selTournament, tournsize=3) # Tournament selection
toolbox.register("mutate", mutate_meal_plan, indpb=0.1)

# Execution Loop
def run_simulation():
    pop = toolbox.population(n=50) # P = 50
    hof = tools.HallOfFame(1) # Elitism: keeps the absolute best chromosome

    # Run simple GA
    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=20,
                        halloffame=hof, verbose=False)

    return hof[0]

In [20]:
# Execute the Simulation
best_plan = run_simulation()

# Calculate Final Nutritional Totals
total_p, total_f, total_c, total_cal = 0.0, 0.0, 0.0, 0.0

print(f"\n{'MEAL':<12} | {'GROUP':<10} | {'FOOD ITEM':<18} | {'SERV.':<6} | {'P (g)':<6} | {'F (g)':<6} | {'C (g)':<6}")
print("-" * 75)

for i, gene in enumerate(best_plan):
    meal = Meals[i // 6]
    group = Food_Groups[i % 6]

    # Get food stats from dataset
    stats = FOOD_DATA[group][gene.food_Item_Name]

    # Calculate totals for this gene
    p = stats['p'] * gene.serving
    f = stats['f'] * gene.serving
    c = stats['c'] * gene.serving
    cal = stats['cal'] * gene.serving

    # Update running totals
    total_p += p
    total_f += f
    total_c += c
    total_cal += cal

    # Print row (only if servings > 0 for clarity)
    if gene.serving > 0:
        print(f"{meal:<12} | {group:<10} | {gene.food_Item_Name:<18} | {gene.serving:<6} | {p:<6.1f} | {f:<6.1f} | {c:<6.1f}")

# Calculate Final Percentages
g_total = total_p + total_f + total_c
p_p = (total_p / g_total) * 100
p_f = (total_f / g_total) * 100
p_c = (total_c / g_total) * 100

# Print Summary Report
print("\n" + "="*40)
print("       DAILY NUTRITION SUMMARY")
print("="*40)
print(f"Total Energy:   {total_cal:.1f} kcal")
print(f"Total Weight:   {g_total:.1f} g (Macro Grams)")
print("-" * 40)
print(f"Nutrient     | Actual % | Target %")
print(f"Carbs (c)    | {p_c:>7.1f}% | 55.0%")
print(f"Protein (p)  | {p_p:>7.1f}% | 20.0%")
print(f"Fats (f)     | {p_f:>7.1f}% | 25.0%")
print("="*40)


MEAL         | GROUP      | FOOD ITEM          | SERV.  | P (g)  | F (g)  | C (g) 
---------------------------------------------------------------------------
Breakfast    | Vegetables | Potato             | 1      | 4.3    | 0.2    | 37.2  
Breakfast    | Grains     | Salted Cracker     | 2      | 0.4    | 1.5    | 3.7   
Breakfast    | Protein    | Omelet             | 1      | 6.5    | 7.3    | 0.4   
Breakfast    | Dairy      | Cheddar Cheese     | 1      | 7.0    | 9.3    | 0.4   
Breakfast    | Fats       | Mayonnaise         | 1      | 0.1    | 4.9    | 3.5   
Snack1       | Fruits     | Blueberries        | 1      | 1.1    | 0.5    | 21.0  
Snack1       | Grains     | Salted Cracker     | 1      | 0.2    | 0.8    | 1.8   
Lunch        | Vegetables | Green Beans        | 1      | 2.0    | 0.1    | 7.8   
Lunch        | Grains     | Salted Cracker     | 3      | 0.7    | 2.3    | 5.5   
Lunch        | Protein    | Omelet             | 2      | 13.0   | 14.6   | 0.8   
Lunch     